In [31]:
from rdflib import Graph
from SPARQLWrapper import SPARQLWrapper, JSON, N3
from pprint import pprint
from pyrdf2vec.graphs import KG
import pandas as pd
import rdflib
from pyrdf2vec.walkers import RandomWalker
from pyrdf2vec.embedders import Word2Vec
from pyrdf2vec import RDF2VecTransformer

In [ ]:
sparql = SPARQLWrapper('https://semopenalex.org/sparql')
sparql.setQuery('''
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX sp: <https://semopenalex.org/property/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX rdf: <http://www.metaphacts.com/ontologies/platform/repository/lookup#>
PREFIX rdfs: <http://www.metaphacts.com/ontologies/repository#>

Select * WHERE {
  	#Alle Works zu Cancer Therapy
	{?work sp:hasConcept <https://semopenalex.org/concept/C2983331546>.
	?work dct:title ?workTitle.
  	?work <http://prismstandard.org/namespaces/basic/2.0/publicationDate> ?workDate.
   #Related Concepts zu allen genannten Works
	?work sp:hasConcept ?relatedConceptsWork.
	?relatedConceptsWork skos:prefLabel ?relatedConceptsWorkTitle.}

}
  	'''
  )
sparql.setReturnFormat(JSON)
qres = sparql.query().convert()
pprint(qres)

In [49]:
knowledge=KG(
    "https://semopenalex.org/sparql",
skip_predicates={"http://prismstandard.org/namespaces/basic/2.0/publicationDate#"}, 
    literals=[
        [
            "https://semopenalex.org/property/class#",
            "http://www.w3.org/2004/02/skos/core#prefLabel",
           
        ],
        [
           
             "https://semopenalex.org/property/hasConcept#",
            "http://www.w3.org/2004/02/skos/core#prefLabel",
        ],
          [
           
             "https://semopenalex.org/property/class#",
            "http://www.w3.org/2004/02/skos/core#prefLabel",
        ],
    ],
),

In [32]:
transformer = RDF2VecTransformer(
    Word2Vec(epochs=10),
    walkers=[RandomWalker(4, 10, with_reverse=False, n_jobs=2)],
    verbose=1
)

In [50]:
embeddings, literals = transformer.fit_transform(knowledge, qres)

AttributeError: 'tuple' object has no attribute 'skip_verify'